In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from joblib import dump, load


In [2]:
#Load node information

spoke = np.load('../../psev_repo/PSEV_matrix')
sep = np.load('../../psev_repo/PSEV_SEP_map')
spoke_node = np.load('../../psev_repo/PSEV_SPOKE_node_map')

spoke = pd.DataFrame(spoke, columns=spoke_node)
spoke.index = sep
spoke.index = spoke.index.map(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
spoke.columns = spoke.columns.map(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

node_type = np.load('../../psev_repo/node_type_list.npy')
node_type = [x.decode('utf-8') if isinstance(x, bytes) else x for x in node_type]
node_type = pd.DataFrame({
    'node': spoke.columns,
    'type': node_type
})

unique_node_types = node_type['type'].unique()

In [3]:
#Translating conditions
# disease_annotation = pd.read_csv('../../psev_repo/omop_sep_map/filtered_omop_conditions_to_spoke_extended_2.tsv', sep = '\t')

disease_annotation = pd.read_csv('../../gbellucci/spoke_linkers/omop2spoke_combined.tsv', sep = '\t')
disease_annotation.rename(columns={'OMOP': 'condition_concept_id'}, inplace=True)
disease_annotation.rename(columns={'SPOKE': 'spoke_concept_id'}, inplace=True)

spoke_to_omop_dict = dict(zip(disease_annotation['spoke_concept_id'], disease_annotation['condition_concept_id']))


#Translating Drugs
drug_annotation = pd.read_csv('../../psev_repo/omop_sep_map/filtered_omop_drug_exposure_to_spoke_extended.tsv', sep = '\t')
drug_annotation.rename(columns={'OMOP': 'condition_concept_id'}, inplace=True)
drug_annotation.rename(columns={'SPOKE': 'spoke_concept_id'}, inplace=True)

spoke_to_omop_dict.update(dict(zip(drug_annotation['spoke_concept_id'], drug_annotation['condition_concept_id'])))


#Translating measurements
lab_annotation = pd.read_csv('../../psev_repo/omop_sep_map/filtered_omop_measurement_to_spoke_extended.tsv', sep = '\t')
lab_annotation.rename(columns={'OMOP': 'condition_concept_id'}, inplace=True)
lab_annotation.rename(columns={'SPOKE': 'spoke_concept_id'}, inplace=True)

spoke_to_omop_dict.update(dict(zip(lab_annotation['spoke_concept_id'], lab_annotation['condition_concept_id'])))


# Assemble the Tables in Pandas

In [4]:
#First load the general top 30% PSEVs

pat_ids = np.load('data/psevs/person_id_index.npy')
columns = np.load('data/psevs/filtered_patient_psevs_columns.npy', allow_pickle=True)
psevs = np.load('data/psevs/filtered_patient_psevs.npy')

full_bio_cohort = pd.read_feather('data/opioid_cohort_details.feather')
label_dict = dict(zip(full_bio_cohort["person_id"], full_bio_cohort["dependent"]))

#Now load the node specific ones

# Initialize empty arrays for columns and psevs
nt_columns = None
nt_psevs = None

for nt in unique_node_types:
    ind_nt_psevs = np.load(f'data/nt_psevs/filtered_patient_psevs_{nt}.npy')
    ind_nt_columns = np.load(f'data/nt_psevs/filtered_patient_psevs_columns_{nt}.npy', allow_pickle=True)

    # Concatenate columns and psevs
    if nt_columns is None:
        nt_columns = ind_nt_columns
    else:
        nt_columns = np.concatenate((nt_columns, ind_nt_columns))  # Add new columns

    if nt_psevs is None:
        nt_psevs = ind_nt_psevs
    else:
        nt_psevs = np.hstack((nt_psevs, ind_nt_psevs))  # Add new data horizontally

In [6]:
columns.shape

(116484,)

In [7]:
nt_columns.shape

(116788,)

In [8]:
#Format tables for RF
Y = np.array([label_dict[pid] for pid in pat_ids if pid in label_dict])
X = psevs
nt_X = nt_psevs

In [9]:
def runGenericRF(X, Y, name):
    # Split data into training and testing sets (80% train, 20% test)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

    # Initialize the Random Forest classifier
    rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)

    # Train the model on the training data
    rf_clf.fit(X_train, Y_train)

    # Predict on the test data
    Y_pred = rf_clf.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(Y_test, Y_pred)
    print(f"{name} Model accuracy: {accuracy:.2f}")

    dump(rf_clf, f'models/addiction_rf_model_{name}.joblib')

    return accuracy, rf_clf

# Train Models

In [ ]:
accuracy_general, model_general = runGenericRF(X, Y, "general")

accuracy_nt, model_nt = runGenericRF(nt_X, Y, "nt")

general Model accuracy: 0.94
nt Model accuracy: 0.94


In [ ]:
# Load here
model_general = load('models/addiction_rf_model_general.joblib')
model_nt = load('models/addiction_rf_model_nt.joblib')

In [22]:
feature_importance_df = pd.DataFrame({
        'feature': columns,
        'importance': model_general.feature_importances_
    })
    
feature_importance_df = pd.merge(feature_importance_df, node_type, left_on = "feature", right_on = "node").drop("feature", axis = 1).sort_values(by = 'importance', ascending = False)
feature_importance_df['omop'] = feature_importance_df['node'].map(spoke_to_omop_dict)

nt_feature_importance_df = pd.DataFrame({
        'feature': nt_columns,
        'importance': model_nt.feature_importances_
    })
    
nt_feature_importance_df = pd.merge(nt_feature_importance_df, node_type, left_on = "feature", right_on = "node").drop("feature", axis = 1).sort_values(by = 'importance', ascending = False)
nt_feature_importance_df['omop'] = nt_feature_importance_df['node'].map(spoke_to_omop_dict)

In [23]:
feature_importance_df.head()

,importance,node,type,omop
114724,0.000663,Q9GZZ6,Protein,NaN
60307,0.000463,CHEMBL267841,Compound,NaN
74229,0.000354,CHEMBL3544968,Compound,NaN
76020,0.000321,CHEMBL368591,Compound,NaN
7170,0.000316,CHEMBL1200703,Compound,NaN


In [24]:
nt_feature_importance_df.head()

,importance,node,type,omop
106589,0.000805,DOID:9975,Disease,436389.0
17631,0.000771,N0000175706,PharmacologicClass,NaN
106193,0.000766,DOID:809,Disease,432303.0
17507,0.000582,N0000006496,PharmacologicClass,3014404.0
2520,0.000561,339778,Gene,NaN


In [13]:
feature_importance_df.groupby('type').count()

,importance,node
type,,
BiologicalProcess,8,8
Compound,106858,106858
Disease,37,37
Gene,40,40
PharmacologicClass,140,140
Protein,9399,9399
Symptom,2,2


In [14]:
nt_feature_importance_df.groupby('type').count()

,importance,node
type,,
Anatomy,3977,3977
BiologicalProcess,3947,3947
CellularComponent,517,517
Compound,86036,86036
Disease,2739,2739
Gene,5870,5870
MolecularFunction,1021,1021
Pathway,729,729
PharmacologicClass,524,524


# Random Forest with ADAboost

In [ ]:
# # Split data into training and testing sets (80% train, 20% test)
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# # Initialize the AdaBoost classifier
# ada_clf = AdaBoostClassifier(n_estimators=50, random_state=42)

# # Train the model on the training data
# ada_clf.fit(X_train, Y_train)

# # Predict on the test data
# Y_pred = ada_clf.predict(X_test)

# # Calculate accuracy
# accuracy = accuracy_score(Y_test, Y_pred)
# print(f"Model accuracy: {accuracy:.2f}")

/pool0/home/sanjay.r/anaconda3/envs/sergio_rotation/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Model accuracy: 0.92


In [ ]:
# dump(ada_clf, 'models/addiction_adaboost_model.joblib')

['addiction_adaboost_model.joblib']

# Load Trained Models

In [ ]:
rf = load('models/addiction_rf_model.joblib')
ada = load('models/addiction_adaboost_model.joblib')
embeddings = pd.read_feather("data/addiction_embeddings.feather")

In [ ]:
feature_importance_df = pd.DataFrame({
        'feature': embeddings.drop('person_id', axis = 1).columns,
        'importance': rf.feature_importances_
    })
    
# Sort by importance and get the top 5 features
top_5_features = feature_importance_df.sort_values(by='importance', ascending=False).head(5)


In [ ]:
feature_importance_df

,feature,importance
0,1,0.0
1,10,0.0
2,100,0.0
3,1000,0.0
4,10000,0.0
...,...,...
389292,X6REB3,0.0
389293,X6REH9,0.0
389294,X6RGC9,0.0
389295,X6RLR1,0.0


In [ ]:
top_5_features

,feature,importance
148557,CHEMBL2172455,0.001193
303852,CHEMBL603053,0.001071
268191,CHEMBL470820,0.000991
240377,CHEMBL383182,0.000929
166629,CHEMBL2348474,0.000851


In [ ]:
#Translating SPOKE to concept 
arr = np.load('../../psev_repo/PSEV_SPOKE_node_map')
arr = np.array([x.decode("utf-8") for x in arr])
arr


array(['1', '10', '100', ..., 'X6RGC9', 'X6RLR1', 'X6RLX0'], dtype='<U14')

In [ ]:
#Translating SPOKE to concept 
arr = np.load('../../psev_repo/node_type_list.npy')
arr = np.array([x.decode("utf-8") for x in arr])
arr


In [ ]:
drug_annotation[drug_annotation['spoke_concept_id'] == 'CHEMBL383182']

,condition_concept_id,spoke_concept_id


In [ ]:
embeddings["CHEMBL383182"]

0       307408.0
1       346975.0
2       302030.0
3       350505.0
4       365222.0
          ...   
6059    363366.0
6060    372583.0
6061    285815.0
6062    349337.0
6063    299372.0
Name: CHEMBL383182, Length: 6064, dtype: float64